#Homework_lecture#6

In [1]:
import statsmodels.stats.multitest as smt
from statsmodels.stats.weightstats import ztest
import pandas as pd
import numpy as np
import scipy.stats as st

Previous function

In [5]:
def check_intervals_intersect(first_ci, second_ci):   
    list_ci = [first_ci, second_ci]                      #collect ci as list
    list_ci.sort()                                       
    if list_ci[1][0] <= list_ci[0][1] <= list_ci[1][1]:  #compare max of smaller ci with bounders of bigger ci
      are_intersect = True
    else:
      are_intersect = False
    return are_intersect                                 # True or False

# function for ci-test 
def check_dge_with_ci(first_table, second_table, alpha):
    ci_test_results = [] 
    for col in first_table.columns[1:-1]:                            #check every genes except last col - Cell Type
      first_ci_gene = st.t.interval(alpha=1-alpha,                   #calculate first ci
                                  df=len(first_table[col]) - 1,
                                  loc=np.mean(first_table[col]),
                                  scale=st.sem(first_table[col]))
      second_ci_gene = st.t.interval(alpha=1-alpha,                  #calculate second ci
                                  df=len(second_table[col]) - 1,
                                  loc=np.mean(second_table[col]),
                                  scale=st.sem(second_table[col]))                 
      ci_test_results.append(check_intervals_intersect(first_ci_gene, second_ci_gene)) #estimate intersection
    return ci_test_results

# function for ztest 
def check_dge_with_ztest(first_table, second_table, alpha):
    z_test_results = []
    for col in first_table.columns[1:-1]:
      ztest_param = ztest(first_table[col], second_table[col])            #ztesting of difference
      z_test_results.append(ztest_param[1] < alpha)                       #compare p-value with threshold
    return z_test_results 

# function for mean differance 
def dif_mean(first_table, second_table):                                        
    dif_mean_results = []
    for col in first_table.columns[1:-1]:
      dif_mean_results.append(np.mean(first_table[col]) - np.mean(second_table[col]))
    return dif_mean_results
    
# function for p-value of z-test
def ztest_pvalue(first_table, second_table):                                    
    ztest_pvalue_results = []
    for col in first_table.columns[1:-1]:
      ztest_pvalue_results.append(ztest(first_table[col], second_table[col])[1])
    return ztest_pvalue_results

New function

In [12]:
def result_p_value_adj(table1, table2, ci_test=True, alpha=.05, multiplete=None):
  b_cells_expression_data = pd.read_csv(f'{table1}',index_col=0)
  nk_cells_expression_data = pd.read_csv(f'{table2}', index_col=0)
  gene_names = b_cells_expression_data.columns[1:-1]
  z_test_results = check_dge_with_ztest(b_cells_expression_data, nk_cells_expression_data, alpha)
  z_test_p_values = ztest_pvalue(b_cells_expression_data, nk_cells_expression_data)
  mean_diff = dif_mean(b_cells_expression_data, nk_cells_expression_data)
  results = {
    "gene": gene_names,
    "mean_diff": mean_diff,
    "z_test_p_values": z_test_p_values,  
  }
  results = pd.DataFrame(results)
  if ci_test:
    ci_test_results = check_dge_with_ci(b_cells_expression_data, nk_cells_expression_data, alpha)
    results.insert(2, "ci_test_results", ci_test_results)
  if multiplete is not None:
    z_test_results_adj = smt.multipletests(z_test_p_values, alpha, multiplete)[0]
    z_test_p_values_adj = smt.multipletests(z_test_p_values, alpha, multiplete)[1]
    results.insert(results.shape[1], "z_test_p_values_adj", z_test_p_values_adj)    
    results.insert(results.shape[1], "z_test_results", z_test_results_adj)
  else:
    results.insert(results.shape[1], "z_test_results", z_test_results)
  return results

In [14]:
result_p_value_adj('b_cells_expression_data.csv', 'nk_cells_expression_data.csv', alpha =.07, multiplete='simes-hochberg')

,gene,mean_diff,ci_test_results,z_test_p_values,z_test_p_values_adj,z_test_results
0,RANBP3,-6.473981,False,0.000063,0.999871,False
1,GABRG3,0.763407,False,0.704603,0.999871,False
2,ARRDC5,-3.334424,False,0.015754,0.999871,False
3,LRP3,2.725465,True,0.091768,0.999871,False
4,TIMM23,-1.074762,True,0.630528,0.999871,False
...,...,...,...,...,...,...
18786,C1orf100,0.071215,False,0.948941,0.999871,False
18787,WDR75,3.675664,True,0.026036,0.999871,False
18788,ZBTB26,1.322516,True,0.366890,0.999871,False
18789,SPTY2D1,-2.393473,True,0.114598,0.999871,False
